In [11]:
from analyze_csv import *
from mutations import get_all_moves
from evaluation_indices import *
from batch_tasks import *
from data_generation import *
from individual import Individual
import numpy as np
import pandas as pd
import os
import traceback
import pickle
from sklearn.decomposition import PCA
from sklearn.metrics import confusion_matrix
from scipy.stats import entropy, skew, kurtosis
from autosklearn.classification import AutoSklearnClassifier
from autosklearn.metrics import f1_micro
from time import time
from random import shuffle

In [2]:
all_moves = np.array(get_all_moves())
all_moves

array(['unguided_merge_gene_move', 'unguided_remove_and_reclassify_move',
       'unguided_split_gene_move', 'expand_cluster_move',
       'split_farthest_move', 'unguided_eliminate_move',
       'knn_reclassification_move', 'one_nth_change_move',
       'centroid_k_means_move', 'centroid_hill_climbing_move',
       'prototype_k_means_move', 'prototype_hill_climbing_move',
       'tree_hill_climbing_move',
       'guided_remove_and_reclassify_move("density_based_validity_separation")',
       'guided_remove_and_reclassify_move("density_based_sparseness_separation")',
       'guided_remove_and_reclassify_move("mean_centroid_distance_separation")',
       'guided_remove_and_reclassify_move("diameter_separation")',
       'guided_histogram_split_gene_move("density_based_validity_separation")',
       'guided_histogram_split_gene_move("density_based_sparseness_separation")',
       'guided_histogram_split_gene_move("mean_centroid_distance_separation")',
       'guided_histogram_split_gene_

In [4]:
index_minimized = {}
for name, func in indices:
    index_minimized[name] = eval(func).is_minimized
index_minimized

{'silhouette': False,
 'calinski_harabaz': False,
 'davies_bouldin': True,
 'dvcb': False,
 'dunn': False,
 'generalized_dunn_41': False,
 'generalized_dunn_43': False,
 'generalized_dunn_51': False,
 'generalized_dunn_53': False,
 'generalized_dunn_13': False}

In [5]:
if True:
    mutation_usage = {}
    def parse_folder(path):
        for idx, filename in enumerate(os.listdir(path)):
            index_name = filename[:filename.find('-')]
            try:
                data, time = read_results(path + '/' + filename)
            except:
                traceback.print_exc()
                continue
            n_clusters = data.iloc[-1]['n_clusters']
            unique_numbers = np.unique(np.bincount(data.index[1:]))
            mutation_names = np.array([s[:s.find(':')] if s is not None and s != 'None' else s for s in data['detail']])
            mut_indices = np.argwhere(mutation_names[:, None] == all_moves[None, :])[:, 1]
            assert len(mut_indices) == len(data) - 1
            data['mutation_index'] = np.hstack([[-1], mut_indices])
            index_series = [t['index'].rename(mutation_names[i] + '_index') for i, t in data[['mutation_index', 'index']].groupby('mutation_index')]
            table = pd.concat(index_series, axis=1).values
            if not index_minimized[index_name]:
                max_indices = np.nanargmax(table, axis=1)
                max_values = np.nanmax(table, axis=1)
                change_steps = np.argwhere(max_values[1:] > max_values[:-1]).flatten() + 1
            else:
                max_indices = np.nanargmin(table, axis=1)
                max_values = np.nanmin(table, axis=1)
                change_steps = np.argwhere(max_values[1:] < max_values[:-1]).flatten() + 1
            c_usage = np.unique(max_indices[change_steps]) - 1
            diff = 0 if filename not in mutation_usage else len(np.setdiff1d(c_usage, mutation_usage[filename])) + len(np.setdiff1d(mutation_usage[filename], c_usage))
            mutation_usage[filename] = c_usage if filename not in mutation_usage else np.unique(np.hstack([c_usage, mutation_usage[filename]]))
            print(idx, filename, n_clusters, len(unique_numbers), len(c_usage), len(change_steps), diff)
            
    parse_folder('/tmp/results-1010')
    parse_folder('/tmp/results2')
    mutation_usage = list(mutation_usage.items())
    with open('mutation_usage.dat', 'wb') as f:
        pickle.dump(mutation_usage, f)
else:
    with open('mutation_usage.dat', 'rb') as f:
        mutation_usage = pickle.load(f)

0 calinski_harabaz-Concrete_Data-one_plus_lambda_all_moves.txt 3 4 7 13 0
1 calinski_harabaz-HappinessRank_2015-one_plus_lambda_all_moves.txt 4 5 2 4 0
2 calinski_harabaz-ICU-one_plus_lambda_all_moves.txt 4 5 3 3 0
3 calinski_harabaz-ParkinsonSpeechDataset-one_plus_lambda_all_moves.txt 4 6 5 13 0
4 calinski_harabaz-analcatdata_lawsuit-one_plus_lambda_all_moves.txt 2 7 3 3 0
5 calinski_harabaz-analcatdata_vineyard-one_plus_lambda_all_moves.txt 2 7 4 8 0
6 calinski_harabaz-analcatdata_wildcat-one_plus_lambda_all_moves.txt 5 6 7 16 0
7 calinski_harabaz-arsenic_female_bladder-one_plus_lambda_all_moves.txt 2 6 6 7 0
8 calinski_harabaz-arsenic_male_bladder-one_plus_lambda_all_moves.txt 3 5 6 8 0
9 calinski_harabaz-arsenic_male_lung-one_plus_lambda_all_moves.txt 2 5 6 6 0
10 calinski_harabaz-autoUniv_au7_1100-one_plus_lambda_all_moves.txt 4 6 5 11 0
11 calinski_harabaz-autoUniv_au7_500-one_plus_lambda_all_moves.txt 4 5 4 4 0
12 calinski_harabaz-autoUniv_au7_700-one_plus_lambda_all_moves.txt 4

108 davies_bouldin-autoUniv_au7_500-one_plus_lambda_all_moves.txt 3 7 3 3 0
109 davies_bouldin-autoUniv_au7_700-one_plus_lambda_all_moves.txt 17 2 10 16 0
110 davies_bouldin-badges2-one_plus_lambda_all_moves.txt 12 2 7 10 0
111 davies_bouldin-banknote_authentication-one_plus_lambda_all_moves.txt 47 2 25 199 0
112 davies_bouldin-boston_corrected-one_plus_lambda_all_moves.txt 3 7 4 5 0
113 davies_bouldin-breast_tissue-one_plus_lambda_all_moves.txt 19 2 8 19 0
114 davies_bouldin-chscase_census2-one_plus_lambda_all_moves.txt 3 7 5 7 0
115 davies_bouldin-chscase_census6-one_plus_lambda_all_moves.txt 3 7 6 7 0
116 davies_bouldin-collins-one_plus_lambda_all_moves.txt 3 8 4 7 0
117 davies_bouldin-dataset_185_yeast-one_plus_lambda_all_moves.txt 7 11 14 19 0
118 davies_bouldin-dataset_187_abalone-one_plus_lambda_all_moves.txt 26 2 13 32 0
119 davies_bouldin-dataset_18_mfeat_morphological-one_plus_lambda_all_moves.txt 41 2 22 184 0
120 davies_bouldin-dataset_2181_auto_price-one_plus_lambda_all_mo

220 dunn-dataset_41_glass-one_plus_lambda_all_moves.txt 4 7 2 2 0
221 dunn-dataset_43_haberman-one_plus_lambda_all_moves.txt 4 7 2 2 0
222 dunn-dataset_53_heart_statlog-one_plus_lambda_all_moves.txt 4 3 3 3 0
223 dunn-dataset_54_vehicle-one_plus_lambda_all_moves.txt 4 5 3 3 0
224 dunn-dataset_8_liver_disorders-one_plus_lambda_all_moves.txt 2 5 1 1 0
225 dunn-debutanizer-one_plus_lambda_all_moves.txt 4 3 6 9 0
226 dunn-disclosure_x_bias-one_plus_lambda_all_moves.txt 8 2 1 1 0
227 dunn-fri_c0_1000_25-one_plus_lambda_all_moves.txt 3 7 3 4 0
228 dunn-fri_c0_100_25-one_plus_lambda_all_moves.txt 3 6 3 3 0
229 dunn-fri_c0_250_10-one_plus_lambda_all_moves.txt 3 6 3 3 0
230 dunn-fri_c1_1000_5-one_plus_lambda_all_moves.txt 3 7 1 4 0
231 dunn-fri_c1_100_10-one_plus_lambda_all_moves.txt 2 6 1 1 0
232 dunn-fri_c1_100_5-one_plus_lambda_all_moves.txt 3 6 1 1 0
233 dunn-fri_c1_250_10-one_plus_lambda_all_moves.txt 3 6 2 2 0
234 dunn-fri_c1_250_5-one_plus_lambda_all_moves.txt 3 7 3 3 0
235 dunn-fri_c1_5

347 dvcb-generated_10dim_10cl-one_plus_lambda_all_moves.txt 7 2 5 12 0
348 dvcb-generated_10dim_30cl-one_plus_lambda_all_moves.txt 18 2 15 19 0
349 dvcb-generated_2dim_10cl-one_plus_lambda_all_moves.txt 9 2 13 23 0
350 dvcb-generated_2dim_30cl-one_plus_lambda_all_moves.txt 7 2 8 25 0
351 dvcb-heart_h-one_plus_lambda_all_moves.txt 4 3 7 8 0
352 dvcb-heart_long_beach-one_plus_lambda_all_moves.txt 4 6 9 14 0
353 dvcb-heart_switzerland-one_plus_lambda_all_moves.txt 4 6 15 20 0
354 dvcb-immunotherapy-one_plus_lambda_all_moves.txt 2 8 9 13 0
355 dvcb-iris-one_plus_lambda_all_moves.txt 5 7 9 10 0
356 dvcb-leaf-one_plus_lambda_all_moves.txt 5 3 7 12 0
357 dvcb-mu284-one_plus_lambda_all_moves.txt 2 8 4 4 0
358 dvcb-oil_spill-one_plus_lambda_all_moves.txt 4 3 13 20 0
359 dvcb-parkinsons-one_plus_lambda_all_moves.txt 4 6 12 16 0
360 dvcb-pathogen_survey_dataset-one_plus_lambda_all_moves.txt 3 7 4 6 0
361 dvcb-planning_relax-one_plus_lambda_all_moves.txt 2 8 4 5 0
362 dvcb-pm10-one_plus_lambda_all

457 generalized_dunn_13-pathogen_survey_dataset-one_plus_lambda_all_moves.txt 3 3 2 2 0
458 generalized_dunn_13-planning_relax-one_plus_lambda_all_moves.txt 3 8 2 3 0
459 generalized_dunn_13-pm10-one_plus_lambda_all_moves.txt 4 7 2 3 0
460 generalized_dunn_13-pollen-one_plus_lambda_all_moves.txt 3 8 6 8 0
461 generalized_dunn_13-prnn_crabs-one_plus_lambda_all_moves.txt 2 4 1 1 0
462 generalized_dunn_13-prnn_fglass-one_plus_lambda_all_moves.txt 4 7 2 2 0
463 generalized_dunn_13-quake-one_plus_lambda_all_moves.txt 4 5 4 6 0
464 generalized_dunn_13-rmftsa_ctoarrivals-one_plus_lambda_all_moves.txt 3 5 3 3 0
465 generalized_dunn_13-rmftsa_sleepdata-one_plus_lambda_all_moves.txt 2 7 4 4 0
466 generalized_dunn_13-sales_transactions-one_plus_lambda_all_moves.txt 144 3 24 118 0
467 generalized_dunn_13-seeds-one_plus_lambda_all_moves.txt 4 4 1 1 0
468 generalized_dunn_13-seismic_bumps-one_plus_lambda_all_moves.txt 2 7 1 1 0
469 generalized_dunn_13-steel_plates_fault-one_plus_lambda_all_moves.txt

...

469 generalized_dunn_13-steel_plates_fault-one_plus_lambda_all_moves.txt 3 7 5 7 7
470 generalized_dunn_13-transplant-one_plus_lambda_all_moves.txt 4 6 2 2 3
471 generalized_dunn_13-user_knowledge-one_plus_lambda_all_moves.txt 4 4 2 2 4
472 generalized_dunn_13-vertebra_column-one_plus_lambda_all_moves.txt 3 6 1 1 9
473 generalized_dunn_13-volcanoes_a2-one_plus_lambda_all_moves.txt 4 4 3 4 4
474 generalized_dunn_13-volcanoes_a3-one_plus_lambda_all_moves.txt 4 6 4 4 6
475 generalized_dunn_13-volcanoes_a4-one_plus_lambda_all_moves.txt 4 4 4 5 10
476 generalized_dunn_13-volcanoes_e1-one_plus_lambda_all_moves.txt 2 6 4 4 4
477 generalized_dunn_13-volcanoes_e2-one_plus_lambda_all_moves.txt 5 2 1 1 2
478 generalized_dunn_13-volcanoes_e3-one_plus_lambda_all_moves.txt 4 4 2 3 4
479 generalized_dunn_13-volcanoes_e4-one_plus_lambda_all_moves.txt 3 2 2 2 3
480 generalized_dunn_13-volcanoes_e5-one_plus_lambda_all_moves.txt 4 4 3 3 6
481 generalized_dunn_13-vowel-one_plus_lambda_all_moves.txt 21 7 9

572 generalized_dunn_41-volcanoes_a4-one_plus_lambda_all_moves.txt 9 3 16 54 11
573 generalized_dunn_41-volcanoes_e1-one_plus_lambda_all_moves.txt 11 3 13 60 10
574 generalized_dunn_41-volcanoes_e2-one_plus_lambda_all_moves.txt 4 6 14 33 11
575 generalized_dunn_41-volcanoes_e3-one_plus_lambda_all_moves.txt 4 6 11 20 8
576 generalized_dunn_41-volcanoes_e4-one_plus_lambda_all_moves.txt 4 6 10 25 7
577 generalized_dunn_41-volcanoes_e5-one_plus_lambda_all_moves.txt 4 5 9 34 7
578 generalized_dunn_41-vowel-one_plus_lambda_all_moves.txt 3 7 9 13 7
579 generalized_dunn_41-wholesale_customers-one_plus_lambda_all_moves.txt 3 6 3 3 9
580 generalized_dunn_41-wine_quality_red-one_plus_lambda_all_moves.txt 2 11 9 11 10
581 generalized_dunn_41-yagoSchema_ttl-one_plus_lambda_all_moves.txt 2 7 2 2 3
582 generalized_dunn_43-Concrete_Data-one_plus_lambda_all_moves.txt 5 4 11 125 11
583 generalized_dunn_43-HappinessRank_2015-one_plus_lambda_all_moves.txt 3 7 2 2 15
584 generalized_dunn_43-ICU-one_plus_la

674 generalized_dunn_43-volcanoes_e5-one_plus_lambda_all_moves.txt 7 3 12 41 10
675 generalized_dunn_43-vowel-one_plus_lambda_all_moves.txt 18 3 25 153 3
676 generalized_dunn_43-wholesale_customers-one_plus_lambda_all_moves.txt 3 7 3 3 3
677 generalized_dunn_43-wine_quality_red-one_plus_lambda_all_moves.txt 3 8 7 8 10
678 generalized_dunn_43-yagoSchema_ttl-one_plus_lambda_all_moves.txt 2 6 2 2 3
679 generalized_dunn_51-Concrete_Data-one_plus_lambda_all_moves.txt 4 4 7 12 8
680 generalized_dunn_51-HappinessRank_2015-one_plus_lambda_all_moves.txt 3 6 2 2 4
681 generalized_dunn_51-ICU-one_plus_lambda_all_moves.txt 3 8 7 10 7
682 generalized_dunn_51-ParkinsonSpeechDataset-one_plus_lambda_all_moves.txt 3 7 5 6 5
683 generalized_dunn_51-analcatdata_lawsuit-one_plus_lambda_all_moves.txt 4 7 4 4 6
684 generalized_dunn_51-analcatdata_vineyard-one_plus_lambda_all_moves.txt 3 7 8 9 10
685 generalized_dunn_51-analcatdata_wildcat-one_plus_lambda_all_moves.txt 4 6 3 3 3
686 generalized_dunn_51-arsen

777 generalized_dunn_53-HappinessRank_2015-one_plus_lambda_all_moves.txt 3 7 4 4 5
778 generalized_dunn_53-ICU-one_plus_lambda_all_moves.txt 4 5 5 7 6
779 generalized_dunn_53-ParkinsonSpeechDataset-one_plus_lambda_all_moves.txt 3 7 9 12 9
780 generalized_dunn_53-analcatdata_lawsuit-one_plus_lambda_all_moves.txt 4 8 12 18 11
781 generalized_dunn_53-analcatdata_vineyard-one_plus_lambda_all_moves.txt 2 7 9 20 4
782 generalized_dunn_53-analcatdata_wildcat-one_plus_lambda_all_moves.txt 3 6 2 3 4
783 generalized_dunn_53-arsenic_female_bladder-one_plus_lambda_all_moves.txt 3 6 12 16 16
784 generalized_dunn_53-arsenic_male_bladder-one_plus_lambda_all_moves.txt 4 8 10 12 10
785 generalized_dunn_53-arsenic_male_lung-one_plus_lambda_all_moves.txt 3 8 7 9 6
786 generalized_dunn_53-autoUniv_au7_1100-one_plus_lambda_all_moves.txt 3 7 11 23 9
787 generalized_dunn_53-autoUniv_au7_500-one_plus_lambda_all_moves.txt 4 5 11 31 2
788 generalized_dunn_53-autoUniv_au7_700-one_plus_lambda_all_moves.txt 4 10 1

880 silhouette-arsenic_female_bladder-one_plus_lambda_all_moves.txt 3 7 5 5 6
881 silhouette-arsenic_male_bladder-one_plus_lambda_all_moves.txt 4 7 5 6 8
882 silhouette-arsenic_male_lung-one_plus_lambda_all_moves.txt 3 7 2 2 3
883 silhouette-autoUniv_au7_1100-one_plus_lambda_all_moves.txt 5 3 6 18 11
884 silhouette-autoUniv_au7_500-one_plus_lambda_all_moves.txt 4 4 5 5 8
885 silhouette-autoUniv_au7_700-one_plus_lambda_all_moves.txt 4 5 5 8 4
886 silhouette-badges2-one_plus_lambda_all_moves.txt 3 7 4 4 2
887 silhouette-banknote_authentication-one_plus_lambda_all_moves.txt 4 6 15 36 6
888 silhouette-boston_corrected-one_plus_lambda_all_moves.txt 3 6 2 2 6
889 silhouette-breast_tissue-one_plus_lambda_all_moves.txt 4 4 3 6 3
890 silhouette-chscase_census2-one_plus_lambda_all_moves.txt 3 6 2 3 5
891 silhouette-chscase_census6-one_plus_lambda_all_moves.txt 3 7 3 4 3
892 silhouette-collins-one_plus_lambda_all_moves.txt 3 6 4 5 4
893 silhouette-dataset_185_yeast-one_plus_lambda_all_moves.txt 4

In [6]:
np.bincount(np.hstack([a[1] for  a in mutation_usage]))

array([462, 519, 254, 427, 246, 549, 265, 336, 461, 224, 469, 399, 388,
       394, 397, 356, 242, 144, 172, 232, 241, 208, 225, 360, 354, 350,
       322, 540, 529, 147,  60])

In [7]:
generated_prefix = 'scratch'
real_prefix = 'datasets'
tasks_list = init_batch(real_prefix)
datasDict = {a: {'dataset': eval(b)} for a, b in datas}
datasDict.keys()

Excluding column number 2 because it is a single value
Excluding column number 23 because it is a single value


dict_keys(['generated_2dim_10cl', 'generated_2dim_30cl', 'generated_10dim_10cl', 'generated_10dim_30cl', 'immunotherapy', 'user_knowledge', 'sales_transactions', 'Concrete_Data', 'HappinessRank_2015', 'ICU', 'ParkinsonSpeechDataset', 'analcatdata_lawsuit', 'analcatdata_vineyard', 'analcatdata_wildcat', 'arsenic_female_bladder', 'arsenic_male_bladder', 'arsenic_male_lung', 'autoUniv_au7_1100', 'autoUniv_au7_500', 'autoUniv_au7_700', 'badges2', 'banknote_authentication', 'boston_corrected', 'breast_tissue', 'chscase_census2', 'chscase_census6', 'collins', 'dataset_185_yeast', 'dataset_187_abalone', 'dataset_18_mfeat_morphological', 'dataset_2181_auto_price', 'dataset_36_segment', 'dataset_39_ecoli', 'dataset_41_glass', 'dataset_43_haberman', 'dataset_53_heart_statlog', 'dataset_54_vehicle', 'dataset_8_liver_disorders', 'debutanizer', 'disclosure_x_bias', 'fri_c0_1000_25', 'fri_c0_100_25', 'fri_c0_250_10', 'fri_c1_1000_5', 'fri_c1_100_10', 'fri_c1_100_5', 'fri_c1_250_10', 'fri_c1_250_5', 

In [8]:
for k, v in datasDict.items():
    dataset = v['dataset']
    v['ma1'] = np.log2(len(dataset))
    v['ma2'] = np.log2(dataset.shape[1])
    meta_attrs = []
    meta_entropy = []
    cont_attrs = []
    for i, s in enumerate(dataset.T):
        uniq = np.unique(s)
        uniq.sort()
        if len(uniq) < len(s) * 0.3:
            meta_attrs.append(i)
            ints = np.searchsorted(uniq, s)
            meta_entropy.append(entropy(ints))
        else:
            cont_attrs.append(i)
    v['ma3'] = len(meta_attrs)
    v['ma5'] = sum(meta_entropy) / len(meta_entropy) if len(meta_entropy) > 0 else 0
    # TODO ma6
    if len(cont_attrs) > 0:
        corr = np.abs(np.corrcoef(dataset[:, cont_attrs], rowvar=False))
        v['ma7'] = corr[np.arange(0, len(corr))[None, :] <= np.arange(0, len(corr))[:, None]].mean() if isinstance(corr, np.ndarray) else corr
        v['ma8'] = np.mean(skew(dataset[:, cont_attrs]))
        v['ma9'] = np.mean(kurtosis(dataset[:, cont_attrs]))
    else:
        v['ma7'], v['ma8'], v['ma9'] = 0, 0, 0
    dists = pdist(dataset)
    v['md1'] = dists.mean()
    v['md2'] = dists.var()
    v['md3'] = dists.std()
    v['md4'] = skew(dists)
    v['md5'] = kurtosis(dists)
    hist = np.histogram(dists, bins=9, range=(0, dists.max()))[0]
    assert len(hist) == 9
    for i, h in enumerate(hist):
        v['md' + str(i + 6)] = h
    z = np.abs(((dists - v['md1']) / v['md3']))
    v['md16'] = (z < 1).sum()
    v['md17'] = ((1 <= z) & (z < 2)).sum()
    v['md18'] = ((2 <= z) & (z < 3)).sum()
    v['md19'] = (3 <= z).sum()

/nix/store/cfr65vfb7crcqb11l1fi384kfgxm3iv1-python3-3.6.8-env/lib/python3.6/site-packages/numpy/lib/function_base.py:2399: RuntimeWarning: invalid value encountered in sqrt
  stddev = sqrt(d.real)


In [9]:
from sklearn.cluster import KMeans, AffinityPropagation, AgglomerativeClustering, MeanShift
state_of_the_art = ['KMeans(n_clusters=n_clusters)', 'AffinityPropagation()', 'AgglomerativeClustering(n_clusters=n_clusters)', 'MeanShift()']

In [10]:
for k, v in datasDict.items():
    start = time()
    dataset = v['dataset']
    n_clusters = int(np.cbrt(len(dataset)))
    values = []
    fnum = 1
    for algo in state_of_the_art:
        try:
            cls = eval(algo).fit_predict(dataset)
        except:
            traceback.print_exc()
            cls = None
        indiv = Individual({'data': dataset, 'labels': cls})
        for _, idxstr in indices:
            try:
                idx = eval(idxstr)(indiv)
            except:
                if cls is not None:
                    traceback.print_exc()
                idx = np.nan
            v['ml' + str(fnum)] = idx
            values.append(idx)
            fnum += 1
    print(k, 'took', time() - start, 'seconds,', values)

/home/dimatomp/Projects/itmo-master-thesis/EvolutionaryClustering/cluster_measures.py:70: RuntimeWarning: divide by zero encountered in power
  cluster_dist = np.where(cluster_dist != 0, cluster_dist ** (-d), 0)
/home/dimatomp/Projects/itmo-master-thesis/EvolutionaryClustering/cluster_measures.py:158: RuntimeWarning: invalid value encountered in true_divide
  result = np.where(matrices.any(axis=1), (cluster_separation - cluster_sparseness) / np.maximum(cluster_separation, cluster_sparseness), -1)
/home/dimatomp/Projects/itmo-master-thesis/EvolutionaryClustering/cluster_measures.py:70: RuntimeWarning: divide by zero encountered in power
  cluster_dist = np.where(cluster_dist != 0, cluster_dist ** (-d), 0)


generated_2dim_10cl took 22.727348804473877 seconds, [0.008740863170556118, 20689.835549467483, 3.513413420143543, 0.043832348645636586, 0.037868448519518635, 0.5359873194599644, 0.12212828007784197, 4.6915405849502685, 1.068998764994415, 0.008628578175253945, 0.4717885917909124, 979.3979928620977, 1.8281120866197924, 0.12884633564087503, 0.0006471633356246015, 0.0006471633356246015, 0.00014777261905393274, 0.0009707450034369022, 0.0002216589285808991, 0.00014777261905393274, 0.7080526112138442, 20532.027486792307, 3.5235245084209232, 0.04490259890482793, 0.08152223388414177, 0.5133660881799031, 0.1169738818874883, 4.523546782766988, 1.0307202584337505, 0.018575383877374514, 0.27952129076914567, 4258.397449231275, 9.372024621639703, 0.273267171822439, 0.5122049534716551, 1.3299988859013268, 0.09272459668541633, 34.57504419086646, 2.4104960251947087, 0.03570980264298902]


/nix/store/cfr65vfb7crcqb11l1fi384kfgxm3iv1-python3-3.6.8-env/lib/python3.6/site-packages/sklearn/metrics/cluster/unsupervised.py:205: RuntimeWarning: invalid value encountered in true_divide
  sil_samples /= np.maximum(intra_clust_dists, inter_clust_dists)
/home/dimatomp/Projects/itmo-master-thesis/EvolutionaryClustering/cluster_measures.py:158: RuntimeWarning: invalid value encountered in true_divide
  result = np.where(matrices.any(axis=1), (cluster_separation - cluster_sparseness) / np.maximum(cluster_separation, cluster_sparseness), -1)
Traceback (most recent call last):
  File "<ipython-input-10-65b6e005b459>", line 16, in <module>
    idx = eval(idxstr)(indiv)
  File "/home/dimatomp/Projects/itmo-master-thesis/EvolutionaryClustering/evaluation_indices.py", line 11, in __call__
    return idx(*args, **kwargs)
  File "/home/dimatomp/Projects/itmo-master-thesis/EvolutionaryClustering/evaluation_indices.py", line 20, in silhouette_index
    return silhouette_score(indiv["data"], ind

generated_2dim_30cl took 18.521546363830566 seconds, [nan, 2252.272322707355, 5.109952215573632, 0.11745088069411688, 0.056676282684684026, 0.7220312012425686, 0.10949359163988173, 4.300294672233534, 0.6521251546503632, 0.008594766737592094, nan, 337.6163226994954, 1.7506957972029402, 0.008541400165164335, 0.005414266524697948, 0.005414266524697948, 0.0010973287492174442, 0.008121399787046922, 0.0016459931238261664, 0.0010973287492174442, nan, 2219.320042355842, 5.459408548171235, 0.15134879749062186, 0.19607973875088028, 0.6763873234364891, 0.11773683768501302, 7.448785157122927, 1.2965890675463905, 0.03413104825404056, nan, nan, 0.0, -0.5090521831735889, nan, nan, nan, nan, nan, nan]


/home/dimatomp/Projects/itmo-master-thesis/EvolutionaryClustering/cluster_measures.py:70: RuntimeWarning: divide by zero encountered in power
  cluster_dist = np.where(cluster_dist != 0, cluster_dist ** (-d), 0)


...

/home/dimatomp/Projects/itmo-master-thesis/EvolutionaryClustering/cluster_measures.py:70: RuntimeWarning: divide by zero encountered in power
  cluster_dist = np.where(cluster_dist != 0, cluster_dist ** (-d), 0)
/home/dimatomp/Projects/itmo-master-thesis/EvolutionaryClustering/cluster_measures.py:158: RuntimeWarning: invalid value encountered in true_divide
  result = np.where(matrices.any(axis=1), (cluster_separation - cluster_sparseness) / np.maximum(cluster_separation, cluster_sparseness), -1)


wholesale_customers took 5.636307239532471 seconds, [0.35441758788050093, 152.0700307462155, 5.701037848396988, -0.32828860353078, 0.03867728183431384, 0.19551078549563766, 0.09424789043550633, 1.6871137579023572, 0.8132897231419725, 0.01864476280130819, 0.24629632086046788, 115.7984684268837, 2.461534053854576, -0.16917536518542978, 0.05561471838904992, 0.25250083044578603, 0.15198752399103096, 1.0933967639073097, 0.6581470112105576, 0.03347610116959606, 0.3854413491724139, 141.68062854437574, 4.948000246649376, -0.3457744654920107, 0.05309368405841414, 0.2432692246591342, 0.09666134616071585, 1.6510116749834123, 0.6560180855369974, 0.021096408643177512, 0.4314361953309489, 39.15060518860407, 3.5027402974655004, -0.27292924126133233, 0.08907906159204873, 0.45924092999783894, 0.05925816001852593, 0.6888613949967584, 0.0888872400277889, 0.011494317995886365]


/home/dimatomp/Projects/itmo-master-thesis/EvolutionaryClustering/cluster_measures.py:70: RuntimeWarning: divide by zero encountered in power
  cluster_dist = np.where(cluster_dist != 0, cluster_dist ** (-d), 0)
/home/dimatomp/Projects/itmo-master-thesis/EvolutionaryClustering/cluster_measures.py:158: RuntimeWarning: invalid value encountered in true_divide
  result = np.where(matrices.any(axis=1), (cluster_separation - cluster_sparseness) / np.maximum(cluster_separation, cluster_sparseness), -1)


wine_quality_red took 19.090579986572266 seconds, [0.1762200491202034, 280.4266630943572, 9.328753037751012, -0.30407330437678215, 0.031046583769834647, 0.11440154145700754, 0.05486722954576435, 1.7740083066250618, 0.850818264649596, 0.014890009493023113, -0.4607898631187821, 9.21832685230387, 143.37897562275444, -0.3681109038788521, 0.000228320917319433, 0.011513233461645516, 0.003015306166138008, 0.5760225123534621, 0.15085981180872005, 5.9797056330444246e-05, 0.12201729206813683, 239.09586388495245, 11.696447559092611, -0.30321581360762717, 0.041189813324156145, 0.10718898454865108, 0.041480748510993586, 1.7128944587097346, 0.6628679669538593, 0.01593992419005144, 0.3609326542053717, 53.90640877208247, 7.432429951391653, -0.307638772796813, 0.05558641654022965, 0.43824706636361105, 0.037768663480979195, 0.7448319731127678, 0.06419052242788886, 0.004790504766732629]
yagoSchema_ttl took 1.057007074356079 seconds, [0.3521478684746755, 200.00355284870713, 3.9296301307838064, -0.44529453

/home/dimatomp/Projects/itmo-master-thesis/EvolutionaryClustering/cluster_measures.py:70: RuntimeWarning: divide by zero encountered in power
  cluster_dist = np.where(cluster_dist != 0, cluster_dist ** (-d), 0)
/home/dimatomp/Projects/itmo-master-thesis/EvolutionaryClustering/cluster_measures.py:158: RuntimeWarning: invalid value encountered in true_divide
  result = np.where(matrices.any(axis=1), (cluster_separation - cluster_sparseness) / np.maximum(cluster_separation, cluster_sparseness), -1)


In [14]:
datanames = list(datasDict)
shuffle(datanames)
cv10 = [datanames[len(datanames)*i//10:len(datanames)*(i+1)//10] for i in range(10)]
dataset_to_cv = {}
for i, c in enumerate(cv10):
    for v in c:
        dataset_to_cv[v] = i

In [16]:
meta_knowledge = [{} for i in range(10)]
def get_meta_list(dataset, name):
    meta_knowledge_list = meta_knowledge[dataset_to_cv[dataset]]
    if name not in meta_knowledge_list:
        meta_knowledge_list[name] = []
    return meta_knowledge_list[name]

In [17]:
for fname, mutations in mutation_usage:
    index, data = fname.split('-')[:2]
    get_meta_list(data, 'index').append(index)
    metadata = datasDict[data]
    for k, v in metadata.items():
        if k == 'dataset': continue
        get_meta_list(data, k).append(v)
    arr = np.zeros(len(all_moves), dtype='bool')
    arr[mutations] = True
    for i, v in enumerate(arr):
        get_meta_list(data, 'rm' + str(i)).append(v)
for d in meta_knowledge:
    assert len(set(map(len, d.values()))) == 1

In [18]:
metaframes = [pd.DataFrame(d) for d in meta_knowledge]
metaframes[0].head()

,index,ma1,ma2,ma3,ma5,ma7,ma8,ma9,md1,md2,...,rm21,rm22,rm23,rm24,rm25,rm26,rm27,rm28,rm29,rm30
0,calinski_harabaz,7.643856,4.321928,17,3.287976,0.522758,0.023865,0.174592,0.142564,0.002888,...,False,False,False,False,False,True,False,False,False,False
1,calinski_harabaz,10.396605,2.321928,1,6.413459,0.655107,-0.122684,0.134221,0.066743,0.000965,...,False,False,False,False,False,True,True,True,False,False
2,calinski_harabaz,8.413628,2.807355,7,5.592515,0.000000,0.000000,0.000000,0.081610,0.002688,...,False,False,False,False,True,False,False,False,True,False
3,calinski_harabaz,9.370687,2.000000,0,0.000000,0.439122,0.349144,0.757310,0.030512,0.000141,...,False,False,False,False,True,True,False,True,False,False
4,calinski_harabaz,7.965784,3.459432,0,0.000000,0.263500,-0.038811,-1.128003,0.290846,0.003762,...,False,False,False,False,True,True,True,False,False,False


In [6]:
train_columns = [c for c in metaframes[0] if not c.startswith('rm')]
test_columns = [c for c in metaframes[0] if c.startswith('rm')]

In [20]:
indices_list = np.array(list(index_minimized))
for d in metaframes:
    d['index'] = np.argwhere(d['index'].values[:, None] == indices_list[None, :])[:, 1].flatten()
    d[test_columns] = d[test_columns].astype('int')
metaframes[0].head()

,index,ma1,ma2,ma3,ma5,ma7,ma8,ma9,md1,md2,...,rm21,rm22,rm23,rm24,rm25,rm26,rm27,rm28,rm29,rm30
0,1,7.643856,4.321928,17,3.287976,0.522758,0.023865,0.174592,0.142564,0.002888,...,0,0,0,0,0,1,0,0,0,0
1,1,10.396605,2.321928,1,6.413459,0.655107,-0.122684,0.134221,0.066743,0.000965,...,0,0,0,0,0,1,1,1,0,0
2,1,8.413628,2.807355,7,5.592515,0.000000,0.000000,0.000000,0.081610,0.002688,...,0,0,0,0,1,0,0,0,1,0
3,1,9.370687,2.000000,0,0.000000,0.439122,0.349144,0.757310,0.030512,0.000141,...,0,0,0,0,1,1,0,1,0,0
4,1,7.965784,3.459432,0,0.000000,0.263500,-0.038811,-1.128003,0.290846,0.003762,...,0,0,0,0,1,1,1,0,0,0


In [22]:
for i, d in enumerate(metaframes):
    d.to_csv('metaframe_{:02d}.csv'.format(i))

In [2]:
metaframes = [pd.read_csv('metaframe_{:02d}.csv'.format(i), index_col=0) for i in range(10)]
metaframes[0].head()

,index,ma1,ma2,ma3,ma5,ma7,ma8,ma9,md1,md2,...,rm21,rm22,rm23,rm24,rm25,rm26,rm27,rm28,rm29,rm30
0,1,7.643856,4.321928,17,3.287976,0.522758,0.023865,0.174592,0.142564,0.002888,...,0,0,0,0,0,1,0,0,0,0
1,1,10.396605,2.321928,1,6.413459,0.655107,-0.122684,0.134221,0.066743,0.000965,...,0,0,0,0,0,1,1,1,0,0
2,1,8.413628,2.807355,7,5.592515,0.000000,0.000000,0.000000,0.081610,0.002688,...,0,0,0,0,1,0,0,0,1,0
3,1,9.370687,2.000000,0,0.000000,0.439122,0.349144,0.757310,0.030512,0.000141,...,0,0,0,0,1,1,0,1,0,0
4,1,7.965784,3.459432,0,0.000000,0.263500,-0.038811,-1.128003,0.290846,0.003762,...,0,0,0,0,1,1,1,0,0,0


In [4]:
metaframe_train = pd.concat(metaframes[:-1])
metaframe_train.head()

,index,ma1,ma2,ma3,ma5,ma7,ma8,ma9,md1,md2,...,rm21,rm22,rm23,rm24,rm25,rm26,rm27,rm28,rm29,rm30
0,1,7.643856,4.321928,17,3.287976,0.522758,0.023865,0.174592,0.142564,0.002888,...,0,0,0,0,0,1,0,0,0,0
1,1,10.396605,2.321928,1,6.413459,0.655107,-0.122684,0.134221,0.066743,0.000965,...,0,0,0,0,0,1,1,1,0,0
2,1,8.413628,2.807355,7,5.592515,0.000000,0.000000,0.000000,0.081610,0.002688,...,0,0,0,0,1,0,0,0,1,0
3,1,9.370687,2.000000,0,0.000000,0.439122,0.349144,0.757310,0.030512,0.000141,...,0,0,0,0,1,1,0,1,0,0
4,1,7.965784,3.459432,0,0.000000,0.263500,-0.038811,-1.128003,0.290846,0.003762,...,0,0,0,0,1,1,1,0,0,0


In [7]:
cls = AutoSklearnClassifier(n_jobs=4)
cls.fit(metaframe_train[train_columns], metaframe_train[test_columns], feat_type=['Categorical'] + ['Numerical'] * (len(train_columns) - 1), metric=f1_micro)

/nix/store/cfr65vfb7crcqb11l1fi384kfgxm3iv1-python3-3.6.8-env/lib/python3.6/site-packages/autosklearn/evaluation/train_evaluator.py:197: RuntimeWarning: Mean of empty slice
  Y_train_pred = np.nanmean(Y_train_pred_full, axis=0)
/nix/store/cfr65vfb7crcqb11l1fi384kfgxm3iv1-python3-3.6.8-env/lib/python3.6/site-packages/autosklearn/evaluation/train_evaluator.py:197: RuntimeWarning: Mean of empty slice
  Y_train_pred = np.nanmean(Y_train_pred_full, axis=0)
/nix/store/cfr65vfb7crcqb11l1fi384kfgxm3iv1-python3-3.6.8-env/lib/python3.6/site-packages/autosklearn/evaluation/train_evaluator.py:197: RuntimeWarning: Mean of empty slice
  Y_train_pred = np.nanmean(Y_train_pred_full, axis=0)
/nix/store/cfr65vfb7crcqb11l1fi384kfgxm3iv1-python3-3.6.8-env/lib/python3.6/site-packages/autosklearn/evaluation/train_evaluator.py:197: RuntimeWarning: Mean of empty slice
  Y_train_pred = np.nanmean(Y_train_pred_full, axis=0)
/nix/store/cfr65vfb7crcqb11l1fi384kfgxm3iv1-python3-3.6.8-env/lib/python3.6/site-package

[WARNING] [2019-05-22 12:20:33,315:EnsembleBuilder(1791095845):39af627e4c2f33e5c297eec0322fbf1c] No models better than random - using Dummy Score!
[WARNING] [2019-05-22 12:20:33,363:EnsembleBuilder(1791095845):39af627e4c2f33e5c297eec0322fbf1c] No models better than random - using Dummy Score!


/nix/store/cfr65vfb7crcqb11l1fi384kfgxm3iv1-python3-3.6.8-env/lib/python3.6/site-packages/autosklearn/evaluation/train_evaluator.py:197: RuntimeWarning: Mean of empty slice
  Y_train_pred = np.nanmean(Y_train_pred_full, axis=0)
/nix/store/cfr65vfb7crcqb11l1fi384kfgxm3iv1-python3-3.6.8-env/lib/python3.6/site-packages/autosklearn/evaluation/train_evaluator.py:197: RuntimeWarning: Mean of empty slice
  Y_train_pred = np.nanmean(Y_train_pred_full, axis=0)
/nix/store/cfr65vfb7crcqb11l1fi384kfgxm3iv1-python3-3.6.8-env/lib/python3.6/site-packages/autosklearn/evaluation/train_evaluator.py:197: RuntimeWarning: Mean of empty slice
  Y_train_pred = np.nanmean(Y_train_pred_full, axis=0)
/nix/store/cfr65vfb7crcqb11l1fi384kfgxm3iv1-python3-3.6.8-env/lib/python3.6/site-packages/autosklearn/evaluation/train_evaluator.py:197: RuntimeWarning: Mean of empty slice
  Y_train_pred = np.nanmean(Y_train_pred_full, axis=0)
/nix/store/cfr65vfb7crcqb11l1fi384kfgxm3iv1-python3-3.6.8-env/lib/python3.6/site-package

/nix/store/cfr65vfb7crcqb11l1fi384kfgxm3iv1-python3-3.6.8-env/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/nix/store/cfr65vfb7crcqb11l1fi384kfgxm3iv1-python3-3.6.8-env/lib/python3.6/site-packages/autosklearn/evaluation/train_evaluator.py:197: RuntimeWarning: Mean of empty slice
  Y_train_pred = np.nanmean(Y_train_pred_full, axis=0)
/nix/store/cfr65vfb7crcqb11l1fi384kfgxm3iv1-python3-3.6.8-env/lib/python3.6/site-packages/autosklearn/evaluation/train_evaluator.py:197: RuntimeWarning: Mean of empty slice
  Y_train_pred = np.nanmean(Y_train_pred_full, axis=0)
/nix/store/cfr65vfb7crcqb11l1fi384kfgxm3iv1-python3-3.6.8-env/lib/python3.6/site-packages/autosklearn/evaluation/train_evaluator.py:197: RuntimeWarning: Mean of empty slice
  Y_train_pred = np.nanmean(Y_train_pred_full, axis=0)
/nix/store/cfr65vfb7crcqb11l1fi384kfgxm3iv

[WARNING] [2019-05-22 12:41:27,157:smac.intensification.intensification.Intensifier] Challenger was the same as the current incumbent; Skipping challenger
[WARNING] [2019-05-22 12:41:27,157:smac.intensification.intensification.Intensifier] Challenger was the same as the current incumbent; Skipping challenger


...

/nix/store/cfr65vfb7crcqb11l1fi384kfgxm3iv1-python3-3.6.8-env/lib/python3.6/site-packages/autosklearn/evaluation/train_evaluator.py:197: RuntimeWarning: Mean of empty slice
  Y_train_pred = np.nanmean(Y_train_pred_full, axis=0)
/nix/store/cfr65vfb7crcqb11l1fi384kfgxm3iv1-python3-3.6.8-env/lib/python3.6/site-packages/autosklearn/evaluation/train_evaluator.py:197: RuntimeWarning: Mean of empty slice
  Y_train_pred = np.nanmean(Y_train_pred_full, axis=0)
/nix/store/cfr65vfb7crcqb11l1fi384kfgxm3iv1-python3-3.6.8-env/lib/python3.6/site-packages/autosklearn/evaluation/train_evaluator.py:197: RuntimeWarning: Mean of empty slice
  Y_train_pred = np.nanmean(Y_train_pred_full, axis=0)
/nix/store/cfr65vfb7crcqb11l1fi384kfgxm3iv1-python3-3.6.8-env/lib/python3.6/site-packages/autosklearn/evaluation/train_evaluator.py:197: RuntimeWarning: Mean of empty slice
  Y_train_pred = np.nanmean(Y_train_pred_full, axis=0)
/nix/store/cfr65vfb7crcqb11l1fi384kfgxm3iv1-python3-3.6.8-env/lib/python3.6/site-package

/nix/store/cfr65vfb7crcqb11l1fi384kfgxm3iv1-python3-3.6.8-env/lib/python3.6/site-packages/autosklearn/evaluation/train_evaluator.py:197: RuntimeWarning: Mean of empty slice
  Y_train_pred = np.nanmean(Y_train_pred_full, axis=0)
/nix/store/cfr65vfb7crcqb11l1fi384kfgxm3iv1-python3-3.6.8-env/lib/python3.6/site-packages/autosklearn/evaluation/train_evaluator.py:197: RuntimeWarning: Mean of empty slice
  Y_train_pred = np.nanmean(Y_train_pred_full, axis=0)
/nix/store/cfr65vfb7crcqb11l1fi384kfgxm3iv1-python3-3.6.8-env/lib/python3.6/site-packages/autosklearn/evaluation/train_evaluator.py:197: RuntimeWarning: Mean of empty slice
  Y_train_pred = np.nanmean(Y_train_pred_full, axis=0)
/nix/store/cfr65vfb7crcqb11l1fi384kfgxm3iv1-python3-3.6.8-env/lib/python3.6/site-packages/autosklearn/evaluation/train_evaluator.py:197: RuntimeWarning: Mean of empty slice
  Y_train_pred = np.nanmean(Y_train_pred_full, axis=0)
/nix/store/cfr65vfb7crcqb11l1fi384kfgxm3iv1-python3-3.6.8-env/lib/python3.6/site-package

[WARNING] [2019-05-22 13:10:17,259:smac.intensification.intensification.Intensifier] Challenger was the same as the current incumbent; Skipping challenger
[WARNING] [2019-05-22 13:10:17,259:smac.intensification.intensification.Intensifier] Challenger was the same as the current incumbent; Skipping challenger


/nix/store/cfr65vfb7crcqb11l1fi384kfgxm3iv1-python3-3.6.8-env/lib/python3.6/site-packages/autosklearn/evaluation/train_evaluator.py:197: RuntimeWarning: Mean of empty slice
  Y_train_pred = np.nanmean(Y_train_pred_full, axis=0)
/nix/store/cfr65vfb7crcqb11l1fi384kfgxm3iv1-python3-3.6.8-env/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/nix/store/cfr65vfb7crcqb11l1fi384kfgxm3iv1-python3-3.6.8-env/lib/python3.6/site-packages/autosklearn/evaluation/train_evaluator.py:197: RuntimeWarning: Mean of empty slice
  Y_train_pred = np.nanmean(Y_train_pred_full, axis=0)
/nix/store/cfr65vfb7crcqb11l1fi384kfgxm3iv1-python3-3.6.8-env/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_f

AutoSklearnClassifier(delete_output_folder_after_terminate=False,
           delete_tmp_folder_after_terminate=False,
           disable_evaluator_output=False, ensemble_memory_limit=1024,
           ensemble_nbest=50, ensemble_size=50, exclude_estimators=None,
           exclude_preprocessors=None, get_smac_object_callback=None,
           include_estimators=None, include_preprocessors=None,
           initial_configurations_via_metalearning=25, logging_config=None,
           metadata_directory=None, ml_memory_limit=3072, n_jobs=4,
           output_folder=None, per_run_time_limit=360,
           resampling_strategy='holdout',
           resampling_strategy_arguments=None, seed=1, shared_mode=False,
           smac_scenario_args=None, time_left_for_this_task=3600,
           tmp_folder=None)

In [9]:
predicted = cls.predict(metaframes[-1][train_columns], n_jobs=4)

In [15]:
confusion_matrix(metaframes[-1][test_columns].values.flatten(), predicted.flatten())

array([[1924,  280],
       [ 390,  506]])

In [27]:
tp_proba = metaframes[-1][test_columns].values.sum() / len(metaframes[-1][test_columns].values.flatten())
random_sample = np.random.choice(2, size=metaframes[-1][test_columns].shape, p=[1-tp_proba, tp_proba])
random_sample.sum(), metaframes[-1][test_columns].values.sum()

(917, 896)